In [82]:
# import sys
# !{sys.executable} -m pip install python_dotenv
# !{sys.executable} -m pip install python_binance
import os
from binance.client import Client
from binance.spot import Spot

import pandas as pd
from dotenv import load_dotenv


In [83]:
# read from local .env file
load_dotenv() 

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_API_SECRET']

client = Client()
spotClient = Spot()
# myClient = Client(api_key, api_secret, testnet=True)

In [84]:
# INPUT
## d2, d3, d4
timeframe = "3d"
## ema (20 or 25)
ema_size = 25
## A symbol with streak >= this will be printed out
streak_threshold = 44


In [85]:
# Function to query Binance for candlestick data
def get_candlestick_data(symbol, timeframe, qty):
    # Retrieve the raw data
    raw_data = spotClient.klines(symbol=symbol, interval=timeframe, limit=qty)
    # Set up the return array
    converted_data = []
    # Convert each element into a Python dictionary object, then add to converted_data
    for candle in raw_data:
        # Dictionary object
        converted_candle = {
            'time': candle[0],
            'open': float(candle[1]),
            'high': float(candle[2]),
            'low': float(candle[3]),
            'close': float(candle[4]),
            'volume': float(candle[5]),
            'close_time': candle[6],
            'quote_asset_volume': float(candle[7]),
            'number_of_trades': int(candle[8]),
            'taker_buy_base_asset_volume': float(candle[9]),
            'taker_buy_quote_asset_volume': float(candle[10])
        }
        # Add to converted_data
        converted_data.append(converted_candle)
    # Return converted data
    return converted_data

In [86]:
# Define function to calculate an arbitrary EMA    
def calc_generic_ema(symbol, timeframe, ema_size):
    raw_data = get_candlestick_data(symbol=symbol, timeframe=timeframe, qty=1000)
    # Convert into Dataframe
    dataframe = pd.DataFrame(raw_data)
    # Create column string
    ema_name = "ema_" + str(ema_size)
    # Create the multiplier
    multiplier = 2/(ema_size + 1)
    # Calculate the initial value (SMA)
    # pd.set_option('display.max_columns', None) # <- use this to show all columns
    # pd.set_option('display.max_rows', None) # <- use this to show all the rows
    initial_mean = dataframe['close'].head(ema_size).mean()

    # Iterate through Dataframe
    for i in range(len(dataframe)):
        if i == ema_size:
            dataframe.loc[i, ema_name] = initial_mean
        elif i > ema_size:
            ema_value = dataframe.loc[i, 'close'] * multiplier + dataframe.loc[i-1, ema_name]*(1-multiplier)
            dataframe.loc[i, ema_name] = ema_value
        else:
            dataframe.loc[i, ema_name] = 0.00
    # print(dataframe) # <- use this to print the dataframe if you want to inspect
    return dataframe

In [87]:
# GET SYMBOL LIST
info = client.get_exchange_info()
usdtList = []
for c in info['symbols']:
    if c['quoteAsset']=='USDT' and c['status']=="TRADING":
        usdtList.append(c['symbol'])
        
print(len(usdtList))
# print(usdtList)
# print(usdtList[1:])

343


In [90]:
# MAIN
subList = usdtList[0:50]
for symbol in subList:
    df = calc_generic_ema(symbol, timeframe, ema_size)

    df['time'] = pd.to_datetime(df['time'], unit='ms')

    df=df.round(2)

    del df['volume']
    del df['close_time']
    del df['quote_asset_volume']
    del df['number_of_trades']
    del df['taker_buy_base_asset_volume']
    del df['taker_buy_quote_asset_volume']

    df['oversell_day'] = 0
    df.loc[df['high'] >= df['ema_25'], 'oversell_day'] = 1


    df['streak'] = df.groupby((df["oversell_day"] != df["oversell_day"].shift()).cumsum()).cumcount()+1
    df.loc[df['oversell_day'] == 0, 'streak'] = 0
    
    if (df['streak'].iloc[-1] > streak_threshold):
        print('Symbol: '+ symbol + ' is having an oversell streak')
        display(df['streak'].iloc[-1])

Symbol: TUSDUSDT is having an oversell streak


411

Symbol: VETUSDT is having an oversell streak


51

Symbol: USDCUSDT is having an oversell streak


...

105

Symbol: IOSTUSDT is having an oversell streak


66

Symbol: WINUSDT is having an oversell streak


440